목적 : 데이터베이스가 실제로 연결 되는지 테스트

#### 1. python 환경 준비 - kernel restart 필요

In [1]:
# pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2_binary-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


#### 2. 데이터베이스 연결

In [ ]:
# # 데이터베이스 연결 설정
# import psycopg2

# try:
#     conn = psycopg2.connect(
#         host = 'localhost',
#         port=5432,
#         database='book_recommend',
#         user='sesac',
#         password='1234'
#     )

#     print('Database connected successfully!')

#     # 커서 생성
#     cursor = conn.cursor()

# except Exception as e:
#     print("An error occurred while connecting to the database:", e)

# finally:
#     if 'conn' in locals() and conn:
#         conn.close()
#         print("Database connection closed.")

Database connected successfully!
Database connection closed.


In [1]:
# 데이터베이스 연결 설정
import psycopg2
from pydantic import BaseModel

class DatabaseSettings(BaseModel):
    host: str = 'localhost'
    port: int = 5432
    database: str = 'book_recommend'
    user: str = 'sesac'
    password: str = '1234'

    print('Database settings successfully!')


# 데이터베이스 연결 함수
def create_connection(settings: DatabaseSettings):
    """데이터베이스 연결 생성"""
    try:
        conn = psycopg2.connect(
            host=settings.host,
            port=settings.port,
            database=settings.database,
            user=settings.user,
            password=settings.password
        )
        print("Database connected successfully!")
        return conn
    except Exception as e:
        print("Database connection failed:", e)
        return None
    
# 데이터베이스 연결
db_settings = DatabaseSettings()
conn = create_connection(db_settings)

Database settings successfully!
Database connected successfully!


In [2]:
# 테이블 생성

def create_tables(conn):
    queries = [
        """
        CREATE TABLE IF NOT EXISTS books (
            isbn VARCHAR(20) PRIMARY KEY,
            title VARCHAR(255) NOT NULL,
            publisher VARCHAR(255),
            author VARCHAR(255),
            description TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS book_highlights (
            isbn VARCHAR(20),
            highlight TEXT NOT NULL,
            FOREIGN KEY (isbn) REFERENCES books (isbn) ON DELETE CASCADE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS tags (
            tag_id SERIAL PRIMARY KEY,
            tag_name VARCHAR(50) UNIQUE NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS book_tags (
            isbn VARCHAR(20),
            tag_id INT,
            PRIMARY KEY (isbn, tag_id),
            FOREIGN KEY (isbn) REFERENCES books (isbn) ON DELETE CASCADE,
            FOREIGN KEY (tag_id) REFERENCES tags (tag_id) ON DELETE CASCADE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS user_choice_tags (
            user_id INT,
            tag_id INT,
            PRIMARY KEY (user_id, tag_id),
            FOREIGN KEY (tag_id) REFERENCES tags (tag_id) ON DELETE CASCADE
        );
        """
    ]


    cursor = None
    try:
        cursor = conn.cursor()
        for query in queries:
            try:
                cursor.execute(query)
                conn.commit()  # 각 쿼리를 개별적으로 커밋
                print("Query executed successfully!")
            except Exception as e:
                conn.rollback()  # 특정 쿼리에서 실패하면 롤백
                print(f"Error executing query: {e}")
    except Exception as e:
        print("Error during table creation process:", e)
    finally:
        cursor.close()

# 테이블 생성 호출
create_tables(conn)

Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!


In [3]:
def insert_sample_data(conn):
    """테이블에 샘플 데이터 삽입"""
    sample_queries = [
        """
        INSERT INTO books (isbn, title, publisher, author, description)
        VALUES
        ('9781234567897', 'Example Book', 'Example Publisher', 'John Doe', 'An example book for testing.')
        ON CONFLICT (isbn) DO NOTHING;
        """,
        """
        INSERT INTO tags (tag_name)
        VALUES
        ('Interesting'), ('Classic'), ('Educational')
        ON CONFLICT (tag_name) DO NOTHING;
        """,
        """
        INSERT INTO book_tags (isbn, tag_id)
        VALUES
        ('9781234567897', 1), ('9781234567897', 2)
        ON CONFLICT DO NOTHING;
        """,
        """
        INSERT INTO user_choice_tags (user_id, tag_id)
        VALUES
        (1, 1), (1, 3)
        ON CONFLICT DO NOTHING;
        """
    ]

    try:
        cursor = conn.cursor()
        for query in sample_queries:
            cursor.execute(query)
        conn.commit()
        print("Sample data inserted successfully!")
    except Exception as e:
        print("Error inserting sample data:", e)
    finally:
        cursor.close()

# 샘플 데이터 삽입 호출
insert_sample_data(conn)

Sample data inserted successfully!


In [4]:
def test_connection(conn):
    """데이터베이스 연결 및 데이터 조회 테스트"""
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM books;")
        rows = cursor.fetchall()
        print("Data in 'books' table:")
        for row in rows:
            print(row)
    except Exception as e:
        print("Error during connection test:", e)
    finally:
        cursor.close()

# 데이터 조회 테스트 호출
test_connection(conn)


Data in 'books' table:
('9781234567897', 'Example Book', 'Example Publisher', 'John Doe', 'An example book for testing.')
